<a href="https://colab.research.google.com/github/mmuokubamichael/5th-place-solution-Lacuna-Correct-Field-Detection-Challenge/blob/main/Lacuna_Correct_Field_Detection_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LACUNA - CORRECT FIELD DETECTION CHALLENGE

This notebook contains the preprocessing step, feature engineering, and modelling for the Lacuna Correct Field Challenge hosted on Zindi.

All Data used was provided by Zindi





Author: Team Proton

### INSTALL MODULES

In [ ]:
#kindly uncomment this if catboost is not installed or if you are running it on google colab

#!pip install catboost

### IMPORT MODULES

In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
from tqdm import tqdm
import skimage.io
from skimage.color import rgb2gray
from skimage.filters import sobel
from tqdm import tqdm
from skimage.filters import threshold_otsu
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from numpy import matrix
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import BaggingRegressor,VotingRegressor
from sklearn.ensemble import AdaBoostRegressor
from catboost import CatBoostRegressor
skf=KFold(n_splits=15)

### READ FILES

In [ ]:
extra_train=pd.read_csv("extra_train.csv")
train_unique=pd.read_csv("train-unique.csv")
auxilary_data=pd.read_csv("auxilary_data-unique.csv")
test=pd.read_csv("test.csv")

### FEATURE GENERATION

The feature generation process used all data provided for Planet Data and Sentinel Data. 

For planet data, the image was loaded from all folders: June 2017, December 2017, June 2018, December 2018, resized, and a threshold applied. The resulting image was then cropped into a 20 by 20 image which was flattened.

For sentinel Data, the image was loaded and a mean of all pixels across all the bands was taken, leaving single values across all bands as features

In [ ]:
dir_extra=["jun17", "dec17","jun18", "dec18"]

In [ ]:
#Load Extra Data

train_list_extra=[]
y_train_extra = []

for index,row in extra_train.iterrows():
    temp_list = []
    for path in tqdm(dir_extra):
        img = os.path.join("extra_train-planet-"+path, row["ID"].split("_")[1]+".png")
        img_array = cv2.imread(img)
        resize_image = cv2.resize(img_array,(83,83))
        x=sobel(rgb2gray(resize_image))
        thresh = threshold_otsu(x)
        binary = x >= thresh
        temp_list.append(binary[30:50, 30:50].flatten())

       
    
    img_2 = os.path.join("extra_train-sentinel", row["ID"].split("_")[1]+".tif")
    img_array_2 = skimage.io.imread(img_2)

    sentinel = list(np.mean(img_array_2, (0,1)))
    if len(sentinel)!=192:
        sentinel = sentinel + [0]*(192 - len(sentinel))

    train_list_extra.append(list((np.array(temp_list)/255).flatten()) + sentinel)
    y_train_extra.append([row["x"],row["y"]])

    

██| 4/4 [00:00<00:00, 148.55it/s]
100%|██████████| 4/4 [00:00<00:00, 148.38it/s]


In [ ]:
#combine initial train data and auxillary Data

combine_train_data=pd.concat([train_unique, auxilary_data],axis=0,ignore_index=True)

In [ ]:
# Load combined data

train_list_combine=[]
y_train_combine = []

for index,row in combine_train_data.iterrows():
    temp_list = []
    for path in tqdm(dir_extra):
        img = os.path.join("planet-"+path, row["ID"].split("_")[1]+".png")
        img_array = cv2.imread(img)
        resize_image = cv2.resize(img_array,(83,83))
        x=sobel(rgb2gray(resize_image))
        thresh = threshold_otsu(x)
        binary = x >= thresh
        temp_list.append(binary[30:50, 30:50].flatten())

        #temp_list = temp_list + list(np.mean(cv2.resize(img_array, (83,83), interpolation = cv2.INTER_AREA), (0,1)))

    img_2 = os.path.join("sentinel", row["ID"].split("_")[1]+".tif")
    img_array_2 = skimage.io.imread(img_2)

    sentinel = list(np.mean(img_array_2, (0,1)))
    if len(sentinel)!=192:
        sentinel = sentinel + [0]*(192 - len(sentinel))

    train_list_extra.append(list((np.array(temp_list)/255).flatten()) + sentinel)
    y_train_combine.append([row["x"],row["y"]])

███| 4/4 [00:00<00:00, 148.55it/s]
100%|██████████| 4/4 [00:00<00:00, 138.16it/s]


In [ ]:
# combine all sets of loaded data

train_list = np.array(train_list_extra + train_list_combine)
y_train = np.array(y_train_extra + y_train_combine)

In [ ]:
# Remove all Zero-Only columns from Array

#train_list = np.delete(train_list, [1613,1614,1629,1630,1645,1646,1661,1662,1677,1678,1693,1694,1709,1710,1725,1726,1741,1742,1757,1758,1773,1774,1789,1790], axis=1)

idx = np.argwhere(np.all(train_list[..., :] == 0, axis=0))

train_list = np.delete(train_list, idx, axis=1)

In [ ]:
#Load Test Data

test_list=[]

for index,row in test.iterrows():
    temp_list = []
    for path in tqdm(dir_extra):
        img = os.path.join("planet-"+path, row["ID"].split("_")[1]+".png")
        img_array = cv2.imread(img)
        resize_image = cv2.resize(img_array,(83,83))
        x=sobel(rgb2gray(resize_image))
        thresh = threshold_otsu(x)
        binary = x >= thresh
        temp_list.append(binary[30:50, 30:50].flatten())

        

    img_2 = os.path.join("sentinel", row["ID"].split("_")[1]+".tif")
    img_array_2 = skimage.io.imread(img_2)

    sentinel = list(np.mean(img_array_2, (0,1)))
    if len(sentinel)!=192:
        sentinel = sentinel + [0]*(192 - len(sentinel))

    test_list.append(list((np.array(temp_list)/255).flatten()) + sentinel)


    
    #y_train_combine.append([row["x"],row["y"]])

██| 4/4 [00:00<00:00, 148.55it/s]
100%|██████████| 4/4 [00:00<00:00, 133.81it/s]


In [ ]:
test_list = np.array(test_list)

In [ ]:
# Remove Zero Only Columns from Test Data

# test_list = np.delete(test_list, [1613,1614,1629,1630,1645,1646,1661,1662,1677,1678,1693,1694,1709,1710,1725,1726,1741,1742,1757,1758,1773,1774,1789,1790], axis=1)


test_list = np.delete(test_list, idx, axis=1)


In [ ]:
test_id = test["ID"]

### Modelling

In [ ]:
ct=CatBoostRegressor(thread_count=2,
                         loss_function='MultiRMSE',
                         eval_metric='MultiRMSE',
                         od_type = 'Iter',
                         verbose= False,
                         bagging_temperature=0.9554779863099283,
                         border_count=179,
                         depth=7,
                         l2_leaf_reg=13,
                         learning_rate=0.02647454924455449,
                         random_strength=1.0014067340536191e-09 ,
                         iterations=422
                     )

In [ ]:
def model_train(estimator,train,label,test):
    mean_train = []
    mean_test_val = []
    test_pred_x = np.zeros(test.shape[0])
    test_pred_y = np.zeros(test.shape[0])
    for count, (train_index,test_index) in enumerate(skf.split(train,label)):
        x_train,x_test = train[train_index],train[test_index]
        y_train,y_test = label[train_index],label[test_index]
        print(f'========================Fold{count +1}==========================')
        estimator.fit(x_train,y_train)
                              
        
        test_pred= estimator.predict(test)
        x=matrix(test_pred).transpose()[0].getA()[0]
        test_pred_x+=x
        y=matrix(test_pred).transpose()[1].getA()[0]
        test_pred_y+=y
    return test_pred_x, test_pred_y

In [ ]:
x,y=model_train(ct,train_list,y_train,test_list)

========================Fold1==========================
========================Fold2==========================
========================Fold3==========================
========================Fold4==========================
========================Fold5==========================
========================Fold6==========================
========================Fold7==========================
========================Fold8==========================
========================Fold9==========================
========================Fold10==========================
========================Fold11==========================
========================Fold12==========================
========================Fold13==========================
========================Fold14==========================
========================Fold15==========================


In [ ]:
pd.DataFrame({'ID': test_id,
'x': x/15,
'y': y/15}).to_csv('sentinel_no_zeros.csv', index=False)